In [3]:
print(layers[0][0].matrice.size())
print(len(crt.tokens))
print(sum([len(ctx.tokens) for ctx in ctxs]))


NameError: name 'layers' is not defined

In [22]:
print(len(crt.tokens))
print([len(ctx.tokens) for ctx in ctxs])
test = Matrice(layers[0][0].matrice[-len(crt.tokens):, -len(crt.tokens):])



25
[36, 44, 33]


Modification des structures pour la lecture du contextes et de l'obtention des chaines de  coréférences correspondantes
Modification de la lecture des phrases de contextes et de la lecture des matrices
Concaténation des matrices sur les lignes
Modification des éléments passés pour l'alignement

In [4]:
print(src)

[['&quot;', 'I', '&apos;m', 'not', 'exag@@', 'ger@@', 'ating', 'when', 'I', 'say', 'that', 'every', 'woman', 'who', 'arri@@', 'ves', 'here', 'has', 'dealt', 'with', 'sexual', 'violence', 'on', 'her', 'way', 'to', 'find', 'refu@@', 'ge', ',', '&quot;', 'said', 'B@@', 'ahr', '.', '<eos>'], ['&quot;', 'Men', 'fle@@', 'e', 'because', 'of', 'war', ',', 'because', 'they', 'are', 'politically', 'persec@@', 'uted', ',', 'because', 'they', 'are', 'threatened', 'with', 'torture', 'or', 'death', ',', 'or', 'because', 'their', 'families', 'pin', 'their', 'hopes', 'on', 'them', 'and', 'send', 'them', 'to', 'Europe', ',', '&quot;', 'she', 'explains', '.', '<eos>'], ['When', 'women', 'fle@@', 'e', ',', 'other', 'reasons', 'are', 'in', 'the', 'fore@@', 'ground', ':', '&quot;', 'Many', 'women', 's@@', 'ne@@', 'ak', 'away', 'secre@@', 'tly', ',', 'because', 'they', 'see', 'no', 'other', 'way', 'out', '.', '&quot;', '<eos>'], ['In', 'their', 'home', 'countries', ',', 'they', 'experienced', 'sexual', 'vio

In [3]:
Matrice.ecriture_xslx(layers[0][0], src, src, "/home/getalp/lopezfab/Documents", "full_matrice")

AttributeError: 'list' object has no attribute 'identifiant'

In [7]:
import Concat_matrice
from Matrice import Matrice
import doctest
from typing import List
from Snt import Snt
import torch
import Utils_data
import Utils_concat
import importlib
importlib.reload(Concat_matrice)
importlib.reload(Utils_data)
importlib.reload(Utils_concat)
doctest.testmod()
torch.set_printoptions(precision=2)
print(f"[DEBUG] Doctest clear")
_DEBUG_START = True
_DEBUG_SUPPR_PAD = True
_DEBUG_NORM_TENSOR = True
_DEBUG_FUSION_BPE= False
_PRECISION = 3
_OUTPUT_PATH=f"/home/getalp/lopezfab/Documents"
id = 1850
r_path=f"/home/getalp/lopezfab/lig/temp/temp/test_attn/{id}.json"
data=Utils_data.lecture_data(r_path)
print(data.keys())
src_seg_lab = data['src_segments_labels']
src = Snt(identifiant=-1, tokens=Utils_concat.ajoute_eos_tokens_src(_snt=data["src_tokens"].split(), src_segments_labels=src_seg_lab))
# src = Utils_concat.ajoute_eos_tokens_src(_snt=data["src_tokens"].split(), src_segments_labels=src_seg_lab)
id = data['id']

# Découpage de la phrase entière en phrase courante + phrases de contexte
src_cutted = Utils_concat.full_sentence_to_ctx_and_crt(src)
ctxs = []
print(f"[DEBUG] len(full_sentence): {len(src)}")
# 
for k in range(len(src_cutted[:-1])):
    ctxs.append(Snt(identifiant=int(data["id"]) - len(src_cutted[k:-1]), tokens = src_cutted[k]))
crt = Snt(identifiant=data['id'], tokens= src_cutted[-1])

# Extraction des différentes matrices à travers les 6 layers et les 8 têtes d'attention de chaque layer
layers = []
for layer in range(len(data['heads_enc_attn'])):
    heads = []
    for head in range(len(data['heads_enc_attn'][layer])):
        full_matrice = torch.tensor(data['heads_enc_attn'][layer][head])
        full_matrice = full_matrice.squeeze()
        heads.append(Matrice(full_matrice))
    layers.append(heads)

# Pour chaque layer, pour chaque tête d'attention, on découpe la matrice en combinaison de k3*k3, k3*k2... k2*k3, k2*k2,... crt*crt
print(layers)
for layer in range(len(layers)):
    for head in range(len(layers[layer])):
        full_matrice_cutted = Utils_concat.cut_matrix_into_sentences(layers[layer][head], src)[-1]

# L x nb_heads x N x N

[DEBUG] Doctest clear
dict_keys(['id', 'src_tokens', 'src_segments_labels', 'tgt_tokens', 'tgt_eos_pos', 'enc_attn', 'dec_attn', 'heads_enc_attn', 'heads_dec_attn'])
[DEBUG] len(full_sentence): 138
[[{'_matrice': tensor([[0.01, 0.01, 0.00,  ..., 0.01, 0.01, 0.02],
        [0.03, 0.02, 0.01,  ..., 0.01, 0.01, 0.01],
        [0.08, 0.10, 0.04,  ..., 0.00, 0.00, 0.00],
        ...,
        [0.00, 0.00, 0.00,  ..., 0.06, 0.02, 0.02],
        [0.00, 0.00, 0.00,  ..., 0.14, 0.09, 0.08],
        [0.00, 0.00, 0.00,  ..., 0.19, 0.09, 0.11]]), '_normalized': False, '_suppr_inf': False}, {'_matrice': tensor([[2.19e-03, 5.78e-02, 5.02e-03,  ..., 9.28e-04, 2.34e-03, 1.09e-03],
        [2.43e-01, 1.38e-02, 2.84e-02,  ..., 5.94e-03, 9.71e-04, 4.67e-04],
        [5.62e-03, 9.57e-01, 5.48e-03,  ..., 7.83e-05, 1.87e-06, 5.20e-07],
        ...,
        [1.70e-07, 6.97e-08, 4.99e-07,  ..., 3.06e-02, 5.04e-04, 9.87e-07],
        [3.83e-07, 8.63e-08, 1.38e-07,  ..., 9.95e-01, 1.86e-04, 5.28e-05],
        [1

TypeError: 'Snt' object is not iterable

# Multi Enc

In [ ]:
import Utils_data as ud
import Utils
import torch
import Matrice
import Sl_matrice
from Snt import Snt
import importlib
for id in [1850]:
    print(f"sentence : {id}")
    precision = 8
    r_path=f"/home/getalp/lopezfab/lig/temp/temp/temp/han_attn2/{id}.json"
    OUTPUT_PATH = f"/home/getalp/lopezfab/Documents/{id}"

    data=ud.lecture_data(r_path)
    crt, ctxs, ctxs_heads, sl_heads = ud.lecture_multi_enc_objet(data)
    mask = torch.ones(sl_heads[0].matrice.shape[1], dtype = torch.bool)
    # Correction dans les cas où il y a moins de 3 contextes
    for k in range(len(ctxs)-1, -1, -1):
        # On supprime les contextes inutiles
        if len(ctxs[k].tokens) == 1 or (len(ctxs[k].tokens) > 1 and ctxs[k].tokens[-2] == "<pad>"):
            del ctxs[k]
            del ctxs_heads[k]
            mask[k] = False
        else:
            # On corrige un problème de padding qui apparait quand il y a moins de 3 contextes
            for h in range(len(ctxs_heads[0])):
                ctxs_heads[k][h].matrice = ctxs_heads[k][h].matrice[..., -len(ctxs[k]):]
                # print(f"[DEBUG]ctxs_heads[k][h].size: {ctxs_heads[k][h].matrice.size()}")

    # S'il y a au moins une phrase de contexte
    if len(ctxs) >= 1:
        for h in range(len(sl_heads)):
            sl_heads[h].matrice = sl_heads[h].matrice[:, mask]

        # print(f"Traitement de la phrase {crt.identifiant}")
        print(crt)
        # print(ctxs[0])
        # print(ctxs[1])
        
        # print(ctxs[2])
        # print(f"len crt : {len(crt)}")
        # print(f"len(ctxs_heads: {len(ctxs_heads)}")
        # print(f"len(ctxs): {[len(ctx) for ctx in ctxs]}")
        # print(f"len(ctxs_heads[0]): {len(ctxs_heads[0])}")
        # print(f"ctxs_heads[0][0].size(): {ctxs_heads[0][0].matrice.size()}")
        # print(f"ctxs_heads[1][0].size(): {ctxs_heads[1][0].matrice.size()}")
        
        # Nettoyage de la phrase courante
        list_crt_suppr_pad = crt.suppr_pad()
        list_crt_fusion_bpe = crt.fusion_bpe()
        # print(f"list_crt_suppr_pad: {list_crt_suppr_pad}")
        # print(f"list_crt_fusion_bpe: {list_crt_fusion_bpe}")
        # print(sl_heads[0].matrice.size())

        mean_ctxs_heads = []
        # Traitement de chaque contexte
        for k in range(len(ctxs)):
            # Nettoyage de la phrases de contexte K
            list_ctx_suppr_pad = ctxs[k].suppr_pad()
            # print(f"len(ctxs[k]) : {len(ctxs[k])}")
            list_ctx_fusion_bpe = ctxs[k].fusion_bpe()
            # print(list_ctx_fusion_bpe)

            for head in range(len(ctxs_heads[k])):
                # Traitement de chaque tête d'attention entre la phrase courante et chaque phrase
                ctxs_heads[k][head].suppr_pad(row_list_suppr_pad= list_crt_suppr_pad, col_list_suppr_pad=  list_ctx_suppr_pad)
                # print(f"size ctxs_heads[k][h] : {ctxs_heads[k][h].matrice.size()}")
                ctxs_heads[k][head].fusion_bpe(row_list_fusion_bpe= list_crt_fusion_bpe, col_list_fusion_bpe= list_ctx_fusion_bpe)
                ctxs_heads[k][head].suppr_inf()
                # ctxs_heads[k][head].norm_tensor()
            # Pour chaque phrase de contexte, on récupère la moyenne des poids d'attention de la phrase courante entre la phrase de contexte (doit être effectuée avant la normalisation )
            mean_ctxs_heads.append(Matrice.Matrice(Utils.mean_matrices([ctxs_heads[k][head].matrice for head in range(len(ctxs_heads[k])) ])))
            mean_ctxs_heads[k].norm_tensor()

        for sl_head in range(len(sl_heads)):
            # Traitement de chaque tête d'attention entre la phrase courante et l'ensemble des phrases de contexte
            sl_heads[sl_head].suppr_pad(row_list_suppr_pad= list_crt_suppr_pad)
            sl_heads[sl_head].fusion_bpe(row_list_fusion_bpe= list_crt_fusion_bpe, action= "mean")
            # sl_heads[sl_head].norm_tensor()
            # sl_heads[sl_head].clean_matrice()

        # On récupère la moyenne des têtes d'attention du mécanisme sentence level
        mean_sl_heads = Sl_matrice.Sl_matrice(Utils.mean_matrices([sl_heads[sl_head].matrice for sl_head in range(len(sl_heads))]))
        mean_sl_heads.norm_tensor()

        # ToDo: Faire la connection entre le mécanisme d'attention word-level et le mécanisme d'attention sentence-level
        full_ctx = ctxs[0] if len(ctxs)>= 1 else None
        if full_ctx is not None:
            for k in range(1, len(ctxs)):
                full_ctx += ctxs[k]


        for k in range(len(ctxs)):
            # Pour chaque phrase de contexte K, 
            # On écrit les matrices de chaque heads entre la phrase  courante et la phrase de contexte K
            # Puis la moyenne de têtes
            for head in range(len(ctxs_heads[k])):
                ctxs_heads[k][head].ecriture_xslx(crt= crt, 
                                                    ctx= ctxs[k],
                                                    absolute_folder= f"{OUTPUT_PATH}/token_level/{head}", 
                                                    filename=f"ctx_{k}_head_{head}", 
                                                    precision=precision, 
                                                    create_folder_path=True)
            mean_ctxs_heads[k].ecriture_xslx(crt= crt,
                                                ctx= ctxs[k],
                                                absolute_folder= f"{OUTPUT_PATH}/token_level", 
                                                filename=f"mean_ctx_{k}", 
                                                create_folder_path=True)

        for sl_head in range(len(sl_heads)):
            # Pour chaque tête d'attention sentence-level,
            # On écrit la tête d'attention entre la phrase courante et les K phrases
            # Puis on écrit la moyenne des têtes d'attention
            # TODO: à vérifier si sl_heads est en mode k3 x k2 x k1 ou k1 x k2 x k3
            # Si sl_heads est en mode  k1 x k2 x k3 alors .flip le passe en mode k3 x k2 x k1 pour plus de lisibilité
            sl_heads[sl_head].matrice=sl_heads[sl_head].matrice.flip(1)
            sl_heads[sl_head].ecriture_xslx(crt= crt, 
                                                absolute_folder= f"{OUTPUT_PATH}/sentence_level", 
                                                filename=f"sl_head_{sl_head}", 
                                                precision=precision, 
                                                create_folder_path=True)
        mean_sl_heads.ecriture_xslx(crt= crt,
                                        absolute_folder= f"{OUTPUT_PATH}/sentence_level", 
                                        filename=f"mean", 
                                        precision=precision, 
                                        create_folder_path=True)

        # Traitement des phrases contextualisé (phrase crt x [k3, k2, k1])
        for h_sl in range(len(sl_heads)):
            for h_tl in range(len(ctxs_heads[0])):
                
                temp = sl_heads[h_sl].contextualise_matrice([ctxs_heads[k][h_tl] for k in range(sl_heads[h_sl].matrice.size(1))])
                # temp.suppr_inf()
                # temp.norm_tensor()
                # print(f"ctxs: {ctxs}")
                
                
                temp.ecriture_xslx(crt= crt,
                                            ctx= full_ctx,
                                            absolute_folder= f"{OUTPUT_PATH}/full_matrice/sl_{h_sl}", 
                                            filename=f"head_{h_tl}", 
                                            precision=8,
                                            create_folder_path=True)


sentence : 1850
dict_keys(['id', 'crt', 'ctxs', 'matrices', 'SL_matrice', 'heads'])
Snt(id=1850, tokens=['<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', 'In', 'their', 'home', 'countries', ',', 'they', 'experienced', 'sexual', 'violence', ',', 'forced', 'marriage', ',', 'honour', 'k@@', 'ill@@', 'ings', ',', 'sla@@', 

## 1 id

In [13]:
import Utils_data as ud
import Utils
import torch
import Matrice
import Sl_matrice
import importlib
id = 1850
precision = 8
r_path=f"/home/getalp/lopezfab/lig/temp/temp/temp/han_attn2/{id}.json"
OUTPUT_PATH = f"/home/getalp/lopezfab/Documents/{id}"

data=ud.lecture_data(r_path)
crt, ctxs, ctxs_heads, sl_heads = ud.lecture_multi_enc_objet(data)

print(f"Traitement de la phrase {crt.identifiant}")
print(ctxs[0])
print(f"len(ctxs_heads: {len(ctxs_heads)}")
print(f"len(ctxs): {[len(ctx) for ctx in ctxs]}")
print(f"len(ctxs_heads[0]): {len(ctxs_heads[0])}")
print(f"ctxs_heads[0][0].size(): {ctxs_heads[0][0].matrice.size()}")
# Nettoyage de la phrase courante
list_crt_suppr_pad = crt.suppr_pad()
list_crt_fusion_bpe = crt.fusion_bpe()
print(f"list_crt_suppr_pad: {list_crt_suppr_pad}")
print(f"list_crt_fusion_bpe: {list_crt_fusion_bpe}")
print(sl_heads[0].matrice.size())

mean_ctxs_heads = []
# Traitement de chaque contexte
for k in range(len(ctxs)):
    # Nettoyage de la phrases de contexte K
    list_ctx_suppr_pad = ctxs[k].suppr_pad()
    list_ctx_fusion_bpe = ctxs[k].fusion_bpe()

    for head in range(len(ctxs_heads[k])):
        # Traitement de chaque tête d'attention entre la phrase courante et chaque phrase
        ctxs_heads[k][head].suppr_pad(row_list_suppr_pad= list_crt_suppr_pad, col_list_suppr_pad=  list_ctx_suppr_pad)
        ctxs_heads[k][head].fusion_bpe(row_list_fusion_bpe= list_crt_fusion_bpe, col_list_fusion_bpe= list_ctx_fusion_bpe)
        # ctxs_heads[k][head].suppr_inf()
        # ctxs_heads[k][head].norm_tensor()
    # Pour chaque phrase de contexte, on récupère la moyenne des poids d'attention de la phrase courante entre la phrase de contexte (doit être effectuée avant la normalisation )
    mean_ctxs_heads.append(Matrice.Matrice(Utils.mean_matrices([ctxs_heads[k][head].matrice for head in range(len(ctxs_heads[k])) ])))
    mean_ctxs_heads[k].norm_tensor()

for sl_head in range(len(sl_heads)):
    # Traitement de chaque tête d'attention entre la phrase courante et l'ensemble des phrases de contexte
    sl_heads[sl_head].suppr_pad(row_list_suppr_pad= list_crt_suppr_pad)
    sl_heads[sl_head].fusion_bpe(row_list_fusion_bpe= list_crt_fusion_bpe, action= "mean")
    # sl_heads[sl_head].norm_tensor()
    # sl_heads[sl_head].clean_matrice()

# On récupère la moyenne des têtes d'attention du mécanisme sentence level
mean_sl_heads = Sl_matrice.Sl_matrice(Utils.mean_matrices([sl_heads[sl_head].matrice for sl_head in range(len(sl_heads))]))
mean_sl_heads.norm_tensor()

# ToDo: Faire la connection entre le mécanisme d'attention word-level et le mécanisme d'attention sentence-level


for k in range(len(ctxs)):
    # Pour chaque phrase de contexte K, 
    # On écrit les matrices de chaque heads entre la phrase  courante et la phrase de contexte K
    # Puis la moyenne de têtes
    for head in range(len(ctxs_heads[k])):
        ctxs_heads[k][head].ecriture_xslx(crt= crt, 
                                            ctx= ctxs[k],
                                            absolute_folder= f"{OUTPUT_PATH}/token_level/{head}", 
                                            filename=f"ctx_{k}_head_{head}", 
                                            precision=precision, 
                                            create_folder_path=True)
    mean_ctxs_heads[k].ecriture_xslx(crt= crt,
                                        ctx= ctxs[k],
                                        absolute_folder= f"{OUTPUT_PATH}/token_level", 
                                        filename=f"mean_ctx_{k}", 
                                        create_folder_path=True)

for sl_head in range(len(sl_heads)):
    # Pour chaque tête d'attention sentence-level,
    # On écrit la tête d'attention entre la phrase courante et les K phrases
    # Puis on écrit la moyenne des têtes d'attention
    # TODO: à vérifier si sl_heads est en mode k3 x k2 x k1 ou k1 x k2 x k3
    # Si sl_heads est en mode  k1 x k2 x k3 alors .flip le passe en mode k3 x k2 x k1 pour plus de lisibilité
    sl_heads[sl_head].matrice=sl_heads[sl_head].matrice.flip(1)
    sl_heads[sl_head].ecriture_xslx(crt= crt, 
                                        absolute_folder= f"{OUTPUT_PATH}/sentence_level", 
                                        filename=f"sl_head_{sl_head}", 
                                        precision=precision, 
                                        create_folder_path=True)
mean_sl_heads.ecriture_xslx(crt= crt,
                                absolute_folder= f"{OUTPUT_PATH}/sentence_level", 
                                filename=f"mean", 
                                precision=precision, 
                                create_folder_path=True)

dict_keys(['id', 'crt', 'ctxs', 'matrices', 'SL_matrice', 'heads'])
Traitement de la phrase 1850
Snt(id=1849, tokens=['<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', 'When', 'women', 'fle@@', 'e', ',', 'other', 'reasons', 'are', 'in', 'the', 'fore@@', 'ground', ':', '&quot;', 'Many', 'women', 's@@', 'ne@@', 'ak', 'away', 'secre@@', 'tly', ',', 'because', 'they', 'see', 'no', 'other', 'way

In [8]:
print(sl_heads[7].matrice[6, 0])
print(ctxs_heads[0][7].matrice[6,14])
print(ctxs_heads[0][7].matrice[6,14] * sl_heads[7].matrice[6, 0])
print(temp.matrice[6, 14])

tensor(0.3367, dtype=torch.float64)
tensor(0.0896, dtype=torch.float64)
tensor(0.0302, dtype=torch.float64)
tensor(0.0302, dtype=torch.float64)


In [ ]:
print(sl_heads[0].matrice.size())
print(ctxs_heads[0][1].matrice.size())
print(ctxs_heads[1][1].matrice.size())
print(ctxs_heads[2][1].matrice.size())
for h in range(len(sl_heads)):
    temp = sl_heads[h].contextualise_matrice([ctxs_heads[0][h], ctxs_heads[1][h], ctxs_heads[2][h]])
    temp.suppr_inf()
    temp.norm_tensor()
    temp.ecriture_xslx(crt= crt,
                                ctx= ctxs[0] + ctxs[1] + ctxs[2],
                                absolute_folder= f"{OUTPUT_PATH}/full_matrice", 
                                filename=f"head_{h}", 
                                precision=8,
                                create_folder_path=True)
print(temp.matrice.size())


torch.Size([23, 3])
torch.Size([23, 29])
torch.Size([23, 43])
torch.Size([23, 32])
[DEBUG] Sl.matrice.__mul__() nb row: 23
[DEBUG] Sl.matrice.__mul__() nb col: 3
test
k : 0.08220548182725906 vs. t: 0.35071104764938354
[DEBUG] full_matrice size : torch.Size([23, 104])
[DEBUG] Création du dossier : /home/getalp/lopezfab/Documents/1850/full_matrice
[DEBUG] Sl.matrice.__mul__() nb row: 23
[DEBUG] Sl.matrice.__mul__() nb col: 3
test
k : 0.05523921549320221 vs. t: 0.3359013497829437
[DEBUG] full_matrice size : torch.Size([23, 104])
[DEBUG] Sl.matrice.__mul__() nb row: 23
[DEBUG] Sl.matrice.__mul__() nb col: 3
test
k : 0.07990622520446777 vs. t: 0.363574355840683
[DEBUG] full_matrice size : torch.Size([23, 104])
[DEBUG] Sl.matrice.__mul__() nb row: 23
[DEBUG] Sl.matrice.__mul__() nb col: 3
test
k : 0.03890309855341911 vs. t: 0.3090135157108307
[DEBUG] full_matrice size : torch.Size([23, 104])
[DEBUG] Sl.matrice.__mul__() nb row: 23
[DEBUG] Sl.matrice.__mul__() nb col: 3
test
k : 0.05947283282

# Save Multienc

In [ ]:
import Utils_data as Utils
import torch
import Matrice
import Sl_matrice
import importlib
for id in range(len(0, 2308)):
    print(f"lecture id : {id}")
    r_path=f"/home/getalp/lopezfab/lig/temp/temp/temp/han_attn2/{id}.json"
    _DEBUG_TRAITEMENT = False
    OUTPUT_PATH = f"/home/getalp/lopezfab/Documents/{id}"

    data=Utils.lecture_data(r_path)
    crt, ctxs, ctxs_heads, sl_heads = Utils.lecture_multi_enc_objet(data)

    # print(f"Traitement de la phrase {crt.identifiant}")
    # print(ctxs[0])
    # print(f"len(ctxs_heads: {len(ctxs_heads)}")
    # print(f"len(ctxs): {[len(ctx) for ctx in ctxs]}")
    # print(f"len(ctxs_heads[0]): {len(ctxs_heads[0])}")
    # print(f"ctxs_heads[0][0].size(): {ctxs_heads[0][0].matrice.size()}")
    print(f" - suppr pad()")
    list_crt_suppr_pad = crt.suppr_pad()
    print(f" - fusion bpe()")
    list_crt_fusion_bpe = crt.fusion_bpe()
    # print(f"list_crt_suppr_pad: {list_crt_suppr_pad}")
    # print(f"list_crt_fusion_bpe: {list_crt_fusion_bpe}")
    # print(sl_heads[0].matrice.size())

    mean_ctxs_heads = []
    for k in range(len(ctxs)):
        if _DEBUG_TRAITEMENT:
            print(f"** Traitement du contexte {k}")
        list_ctx_suppr_pad = ctxs[k].suppr_pad()
        list_ctx_fusion_bpe = ctxs[k].fusion_bpe()

        for head in range(len(ctxs_heads[k])):
            if _DEBUG_TRAITEMENT:
                print(f"   traitement de la head: {head}")
            ctxs_heads[k][head].suppr_pad(row_list_suppr_pad= list_crt_suppr_pad, col_list_suppr_pad=  list_ctx_suppr_pad)
            ctxs_heads[k][head].fusion_bpe(row_list_fusion_bpe= list_crt_fusion_bpe, col_list_fusion_bpe= list_ctx_fusion_bpe)
            ctxs_heads[k][head].clean_matrice()
            ctxs_heads[k][head].norm_tensor()
        mean_ctxs_heads.append(Matrice.Matrice(Utils.mean_matrices([ctxs_heads[k][head].matrice for head in range(len(ctxs_heads[k])) ])))
        mean_ctxs_heads[k].norm_tensor()

    for sl_head in range(len(sl_heads)):
        if _DEBUG_TRAITEMENT:
            print(f"   traitement de la sl_head: {sl_head}")
        sl_heads[sl_head].suppr_pad(row_list_suppr_pad= list_crt_suppr_pad)
        sl_heads[sl_head].fusion_bpe(row_list_fusion_bpe= list_crt_fusion_bpe, action= torch.mean)
        sl_heads[sl_head].norm_tensor()
        # sl_heads[sl_head].clean_matrice()
    mean_sl_heads = Sl_matrice.Sl_matrice(Utils.mean_matrices([sl_heads[sl_head].matrice for sl_head in range(len(sl_heads))]))
    mean_sl_heads.norm_tensor()


    for k in range(len(ctxs)):
        # print(f"écriture du contexte {k}")
        for head in range(len(ctxs_heads[k])):
            # print(f"écriture de la head {head}")

            ctxs_heads[k][head].ecriture_xslx(crt= crt, 
                                                ctx= ctxs[k],
                                                absolute_folder= f"{OUTPUT_PATH}/token_level/{head}", 
                                                filename=f"ctx_{k}_head_{head}", 
                                                create_folder_path=True)
        mean_ctxs_heads[k].ecriture_xslx(crt= crt,
                                            ctx= ctxs[k],
                                            absolute_folder= f"{OUTPUT_PATH}/token_level", 
                                            filename=f"mean_ctx_{k}", 
                                            create_folder_path=True)
    for sl_head in range(len(sl_heads)):
        # print(f"écriture de la sl_head {sl_head}")
        sl_heads[sl_head].ecriture_xslx(crt= crt, 
                                            absolute_folder= f"{OUTPUT_PATH}/sentence_level/{sl_head}", 
                                            filename=f"sl_head{sl_head}", 
                                            create_folder_path=True)
    mean_sl_heads.ecriture_xslx(crt= crt,
                                    absolute_folder= f"{OUTPUT_PATH}/sentence_level", 
                                    filename=f"mean_sl_head", 
                                    create_folder_path=True)